In [2]:
import cantera as ct 
import cantera.ck2cti
from rmgpy import chemkin
import rmgpy.chemkin
from rmgpy import kinetics
import numpy as np
import subprocess
import csv
import copy
import os

# Objective

- From the following experiments, we know that there are several important reactions to focus on in Nora's model
    - start with "bad" model, with 129 species added in core
    - at 130 species, add in **C#CC(5272) <=> H(8) + C3H3(6822)** which "fixes" speed
    - at 133 species, several reactions added in to make it worse again:
        - H(8) + C3H3(6822) <=> CH2(S)(25) + C2H2(23)
        - H(8) + C3H3(6822) <=> CH3(19) + C2H(22)
        

In [5]:
# Load the models
# load NIST 2-BTP model, with all the halogens 
#load Nora's 2-BTP model, with all the halogens 

# Load Nora's model (145 species)

full_path_Nora = '/work/westgroup/nora/Code/projects/halogens/refrigerants/singles/Burgess_Comments/cantera/Nora/2_BTP/FFCM_seed/2_BTP_seed/chemkin/'

Nora_chemkin_path = full_path_Nora + 'copies/copy_chem0145.inp'
Nora_dictionary_path = full_path_Nora + 'species_dictionary.txt'
Nora_transport_path = full_path_Nora + 'tran.dat'
Nora_cti_path = full_path_Nora + 'copies/copy_chem0145.cti'

Nora_species_list, Nora_reaction_list = rmgpy.chemkin.load_chemkin_file(Nora_chemkin_path, dictionary_path=Nora_dictionary_path, transport_path=Nora_transport_path)
Nora_gas = ct.Solution(Nora_cti_path)


# Load full NIST model

full_path_NIST = '/work/westgroup/nora/Code/projects/halogens/refrigerants/singles/Burgess_Comments/cantera/difftool/NIST/'


NIST_cti_path = full_path_NIST + '2-BTP_kinetics.cti'
NIST_chemkin_path = full_path_NIST + 'pym_gas_copy.inp'
NIST_dictionary_path = full_path_NIST + 'species_dictionary_2-BTP.txt'

NIST_gas = ct.Solution(NIST_cti_path)
NIST_dict = rmgpy.chemkin.load_species_dictionary(NIST_dictionary_path)
NIST_species_list, NIST_reaction_list = rmgpy.chemkin.load_chemkin_file(NIST_chemkin_path, dictionary_path=NIST_dictionary_path)



**** WARNING ****
For species CH2OCH, discontinuity in cp/R detected at Tmid = 500
	Value computed using low-temperature polynomial:  8.39347
	Value computed using high-temperature polynomial: 9.1801


**** WARNING ****
For species CH2OCH, discontinuity in h/RT detected at Tmid = 500
	Value computed using low-temperature polynomial:  42.1991
	Value computed using high-temperature polynomial: 41.9615


**** WARNING ****
For species CH2OCH, discontinuity in s/R detected at Tmid = 500
	Value computed using low-temperature polynomial:  33.7069
	Value computed using high-temperature polynomial: 33.5121


**** WARNING ****
For species C4H5-2, discontinuity in h/RT detected at Tmid = 1000
	Value computed using low-temperature polynomial:  47.6524
	Value computed using high-temperature polynomial: 48.4362


**** WARNING ****
For species C4H5-2, discontinuity in s/R detected at Tmid = 1000
	Value computed using low-temperature polynomial:  52.4292
	Value computed using high-temperature polyno

In [144]:
Nora_reactions_with_C3H3 = [] 
for rxn in Nora_reaction_list:
    for spc in rxn.products:
        if str(spc) == 'C3H3(6822)':
            Nora_reactions_with_C3H3.append(rxn)
    for spc in rxn.reactants:
        if str(spc) == 'C3H3(6822)':
            Nora_reactions_with_C3H3.append(rxn)

# len(Nora_reactions_with_C3H3:
for rxn in Nora_reactions_with_C3H3:
     print(rxn)
#str(Nora_reactions_with_C3H3[2])


H2O2(14) + C3H3(6822) <=> HO2(13) + C#CC(5272)
HCO(17) + C3H3(6822) <=> CO(15) + C#CC(5272)
CHO2(230) + C3H3(6822) <=> CO2(16) + C#CC(5272)
CH4(3) + C3H3(6822) <=> CH3(19) + C#CC(5272)
CH2OH(33) + C3H3(6822) <=> CH2O(20) + C#CC(5272)
CH3O(27) + C3H3(6822) <=> CH2O(20) + C#CC(5272)
C2H3(29) + C3H3(6822) <=> C2H2(23) + C#CC(5272)
CH3CO(34) + C3H3(6822) <=> CH2CO(28) + C#CC(5272)
CH2CHO(35) + C3H3(6822) <=> CH2CO(28) + C#CC(5272)
C2H5(32) + C3H3(6822) <=> C2H4(30) + C#CC(5272)
C3H3(6822) + S(140) <=> 2-BTP(1) + C#CC(5272)
HO2(13) + C3H3(6822) <=> O2(4) + C#CC(5272)
S(641) + C3H3(6822) <=> CF3CCH(84) + C#CC(5272)
S(127) + C3H3(6822) <=> CF3CCH(84) + C#CC(5272)
HBR(92) + C3H3(6822) <=> BR(90) + C#CC(5272)
C3H3(6822) + S(427) <=> C#CC(5272) + S(1838)
CH2Br(969) + C#CC(5272) <=> CBr(425) + C3H3(6822)
H(8) + C3H3(6822) <=> C#CC(5272)
HO2(13) + C3H3(6822) <=> O2(157) + C#CC(5272)
CF3(45) + C#CC(5272) <=> CHF3(42) + C3H3(6822)
H(8) + C3H3(6822) <=> C3H4(6823)
CF3(45) + C3H3(6822) <=> S(4579)
CO2

In [145]:
for idx,rxn in enumerate(Nora_reaction_list):
    if str(rxn)== 'H(8) + C3H3(6822) <=> C#CC(5272)':
        print(rxn)
        print(idx)
        print(rxn.index)

H(8) + C3H3(6822) <=> C#CC(5272)
1271
1285


In [133]:
for idx, rxn in enumerate(NIST_reaction_list):
    if str(rxn) == 'C3H3 + H <=> pC3H4':
        print(rxn)
        print(idx)
        print(rxn.index)
# for i in NIST_reactions_with_C3H3:
#     print(i)

C3H3 + H <=> pC3H4
276
284


# Report

- From the following experiments, we know that there are several important reactions to focus on in Nora's model
    - start with "bad" model, with 129 species added in core
    - at 130 species, add in C#CC(5272) <=> H(8) + C3H3(6822) which "fixes" speed
    - at 133 species, several reactions added in to make it worse again:
            - H(8) + C3H3(6822) <=> CH2(S)(25) + C2H2(23)
            - H(8) + C3H3(6822) <=> CH3(19) + C2H(22)

- In the different models
    - (NIST).......(RMG).......... Equation
    -  285 ........ 1287............. H(8) + C3H3(6822) <=> C#CC(5272) ..............................| C3H3 + H <=> pC3H4    
    -  164 ........ 1321............. H(8) + C3H3(6822) <=> CH2(S)(25) + C2H2(23) .............|  C2H2 + CH2 <=> C3H3 + H
    -  107 ........ 1322............. H(8) + C3H3(6822) <=> CH3(19) + C2H(22)................... |   C2H + CH3 <=> C3H3 + H


Difference in kinetics between these reactions: 

                            NIST                         RMG

- C3H3 + H <=> pC3H4
           type           Arrhenius                    Chebyshev
           press dep?       False                        True 
 
- C2H2 + CH2 <=> C3H3 + H
           type           Arrhenius                    Chebyshev   
           press dep?       False                        True 
           
- C3H3 + H <=> pC3H4
           type           Arrhenius                    Chebyshev   
           press dep?       False                        True 

In [119]:
Nora_reaction_list[156]

In [146]:

# C2H + CH3 <=> C3H3 + H

ind= [276]

for i in ind: 
    print(type((NIST_reaction_list[i].kinetics)))
    print(NIST_reaction_list[i].kinetics.is_pressure_dependent())

    try:
        print(NIST_reaction_list[i])
    except: 
        pass
    try:
        print(NIST_reaction_list[i].kinetics._A)
    except: 
        pass
    try:
        print(NIST_reaction_list[i].kinetics._Ea)
    except: 
        pass
    try:
        print(NIST_reaction_list[i].kinetics._n)
    except: 
        pass

<class 'rmgpy.kinetics.arrhenius.Arrhenius'>
False
C3H3 + H <=> pC3H4
1.5e+13 cm^3/(mol*s)
0 cal/mol
0


In [147]:
ind= [1271]

for i in ind:
    print(type(Nora_reaction_list[i].kinetics))
    print(Nora_reaction_list[i].kinetics.is_pressure_dependent())
    try:
        print(Nora_reaction_list[i])
    except: 
        pass
    try:
        print(Nora_reaction_list[i].kinetics._A)
    except: 
        pass
    try:
        print(Nora_reaction_list[i].kinetics._Ea)
    except: 
        pass
    try:
        print(Nora_reaction_list[i].kinetics._n)
    except: 
        pass

Nora_reaction_list[1308].kinetics.chebyshev


<class 'rmgpy.kinetics.chebyshev.Chebyshev'>
True
H(8) + C3H3(6822) <=> C#CC(5272)


<bound method Chebyshev.chebyshev of Chebyshev(coeffs=[[1.687,-0.1725,-0.1046,-0.04468],[10.9,0.1404,0.08197,0.03187],[-0.1159,0.0273,0.01844,0.009661],[-0.04875,-0.0008685,0.0002676,0.0009002],[-0.01749,-0.004785,-0.002774,-0.001052],[-0.006025,-0.002157,-0.001435,-0.0007333]], kunits='cm^3/(mol*s)', Tmin=(300,'K'), Tmax=(2500,'K'), Pmin=(0.01,'atm'), Pmax=(98.692,'atm'))>

In [109]:
dir(Nora_reaction_list[1308].kinetics)

['Pmax',
 'Pmin',
 'Tmax',
 'Tmin',
 '_Pmax',
 '_Pmin',
 '_Tmax',
 '_Tmin',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__pyx_vtable__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '_coeffs',
 'change_rate',
 'chebyshev',
 'coeffs',
 'comment',
 'degreeP',
 'degreeT',
 'discrepancy',
 'efficiencies',
 'fit_to_data',
 'get_cantera_efficiencies',
 'get_effective_collider_efficiencies',
 'get_effective_pressure',
 'get_rate_coefficient',
 'get_reduced_pressure',
 'get_reduced_temperature',
 'highPlimit',
 'is_identical_to',
 'is_pressure_dependent',
 'is_pressure_valid',
 'is_similar_to',
 'is_temperature_valid',
 'kunits',
 'set_cantera_kinetics',
 'to_html',
 'uncertainty']